TELEGRAM_TOKEN = "8122795159:AAG1wSVngJljUC8QUPfyBlGj0_7X-qJyi5Q"
TMDB_API_KEY = "6000ee7cd355e82c5e7961907b0f7d11"

In [3]:
import logging
import requests
import threading
import sqlite3
import re

from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update
from telegram.ext import (
    Updater,
    CommandHandler,
    MessageHandler,
    Filters,
    CallbackQueryHandler,
    CallbackContext
)

logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

TELEGRAM_TOKEN = "8122795159:AAG1wSVngJljUC8QUPfyBlGj0_7X-qJyi5Q"
TMDB_API_KEY = "6000ee7cd355e82c5e7961907b0f7d11"

TMDB_BASE_URL = "https://api.themoviedb.org/3"
IMAGE_BASE_URL = "https://image.tmdb.org/t/p/w500"


# ------------------------------------------------------------------------------
# 0. ИНИЦИАЛИЗАЦИЯ БАЗЫ
# ------------------------------------------------------------------------------
def init_db():
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS user_movies (
            user_id INTEGER,
            movie_id INTEGER
        )
    ''')
    conn.commit()
    conn.close()

def add_watched_movie(user_id, movie_id):
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute('SELECT 1 FROM user_movies WHERE user_id=? AND movie_id=?',
                   (user_id, movie_id))
    row = cursor.fetchone()
    if not row:
        cursor.execute('INSERT INTO user_movies (user_id, movie_id) VALUES (?, ?)',
                       (user_id, movie_id))
        conn.commit()
    conn.close()

def get_user_movies(user_id):
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute('SELECT movie_id FROM user_movies WHERE user_id=?', (user_id,))
    rows = cursor.fetchall()
    conn.close()
    return [r[0] for r in rows]

def clear_user_movies(user_id):
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute('DELETE FROM user_movies WHERE user_id=?', (user_id,))
    conn.commit()
    conn.close()

def user_has_watched_movie(user_id, movie_id):
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute('SELECT 1 FROM user_movies WHERE user_id=? AND movie_id=?',
                   (user_id, movie_id))
    row = cursor.fetchone()
    conn.close()
    return row is not None


# ------------------------------------------------------------------------------
# 1. Основные команды
# ------------------------------------------------------------------------------
def start_command(update, context):
    update.message.reply_text(
        "Привет! Я бот для поиска фильмов.\n\n"
        "Команды:\n"
        "/search_title <название> — поиск фильмов по названию.\n"
        "/search_genre <жанр> [год или диапазон годов] — поиск по жанру.\n"
        "/similar_title <название> — найти фильмы, похожие на указанный.\n"
        "/my_watched — показать все просмотренные фильмы.\n"
        "/clear_watched — очистить список просмотренных.\n\n"
        "Например:\n"
        "/search_title Аватар\n"
        "/search_genre комедия 2019-2020\n"
        "/similar_title Аватар"
    )

def search_title_command(update, context):
    user_id = update.effective_user.id
    user_title = " ".join(context.args)
    if not user_title:
        update.message.reply_text("Введите название фильма. Пример: /search_title Аватар")
        return

    try:
        all_movies = get_movies_by_title(user_title)
        filtered_movies = [m for m in all_movies if not user_has_watched_movie(user_id, m['id'])]

        if filtered_movies:
            for m in filtered_movies[:5]:
                send_movie_info(update, m)
        else:
            if all_movies:
                update.message.reply_text("Все найденные фильмы уже были вами просмотрены!")
            else:
                update.message.reply_text(f"По названию '{user_title}' ничего не найдено.")
    except Exception as e:
        logger.error(f"Ошибка при поиске по названию: {e}")
        update.message.reply_text("Произошла ошибка при поиске. Попробуйте ещё раз.")

def my_watched_command(update, context):
    user_id = update.effective_user.id
    watched_ids = get_user_movies(user_id)
    if not watched_ids:
        update.message.reply_text("У вас нет просмотренных фильмов.")
        return

    lines = []
    for movie_id in watched_ids:
        movie_data = get_movie_details(movie_id)
        if movie_data:
            title = movie_data.get("title", "Без названия")
            release_date = movie_data.get("release_date", "N/A")
            line = f"• {title} ({release_date})"
            lines.append(line)
        else:
            lines.append(f"• [ID={movie_id}] Не удалось получить данные.")

    full_text = "Ваши просмотренные фильмы:\n\n" + "\n".join(lines)
    update.message.reply_text(full_text)

def clear_watched_command(update, context):
    user_id = update.effective_user.id
    clear_user_movies(user_id)
    update.message.reply_text("Список просмотренных фильмов очищен!")


# ------------------------------------------------------------------------------
# 2. Поиск по жанру (с указанием года / интервала)
# ------------------------------------------------------------------------------
def search_genre_command(update, context):
    user_id = update.effective_user.id
    user_args = context.args

    if not user_args:
        update.message.reply_text("Введите жанр. Пример: /search_genre комедия 2015-2017")
        return

    possible_year_arg = user_args[-1]
    years_range = None

    if re.match(r"^\d{4}-\d{4}$", possible_year_arg):
        start_year_str, end_year_str = possible_year_arg.split("-")
        years_range = (start_year_str, end_year_str)
        user_genre_parts = user_args[:-1]
    elif re.match(r"^\d{4}$", possible_year_arg):
        start_year_str = possible_year_arg
        end_year_str = possible_year_arg
        years_range = (start_year_str, end_year_str)
        user_genre_parts = user_args[:-1]
    else:
        user_genre_parts = user_args

    user_genre = " ".join(user_genre_parts).strip()
    if not user_genre:
        update.message.reply_text("Пожалуйста, укажите жанр. Пример: /search_genre комедия 2020")
        return

    try:
        all_movies = get_movies_by_genre(user_genre, years_range=years_range)
        filtered_movies = [m for m in all_movies if not user_has_watched_movie(user_id, m['id'])]

        if filtered_movies:
            context.user_data["movies"] = filtered_movies
            context.user_data["next_index"] = 0
            show_more_movies(update, context)
        else:
            if all_movies:
                update.message.reply_text("Все найденные фильмы по этому жанру вы уже посмотрели!")
            else:
                update.message.reply_text(f"По жанру '{user_genre}' ничего не найдено (с учётом года).")
    except Exception as e:
        logger.error(f"Ошибка при поиске по жанру: {e}")
        update.message.reply_text("Произошла ошибка при поиске. Попробуйте ещё раз.")


# ------------------------------------------------------------------------------
# 2a. НОВАЯ КОМАНДА: /similar_title (с «Показать ещё» на 3 фильма)
# ------------------------------------------------------------------------------
def similar_title_command(update, context):
    """
    /similar_title <название>

    1) Находим фильм по названию (search/movie).
    2) Берём первый из найденных.
    3) Запрашиваем похожие.
    4) Фильтруем просмотренные.
    5) Сохраняем в context.user_data и показываем первые 3 с кнопкой «Показать ещё».
    """
    user_id = update.effective_user.id
    user_title = " ".join(context.args)
    if not user_title:
        update.message.reply_text("Введите название фильма. Пример: /similar_title Аватар")
        return

    try:
        found = get_movies_by_title(user_title)
        if not found:
            update.message.reply_text(f"Не найдено фильмов с названием '{user_title}'.")
            return

        base_movie = found[0]
        base_id = base_movie.get("id")
        base_title = base_movie.get("title", "Без названия")

        similar = get_similar_movies(base_id)
        if not similar:
            update.message.reply_text(f"Не найдены похожие фильмы на '{base_title}'.")
            return

        filtered = [m for m in similar if not user_has_watched_movie(user_id, m["id"])]

        if filtered:
            # Сохраняем список похожих в user_data
            context.user_data["similar_movies"] = filtered
            context.user_data["similar_index"] = 0
            context.user_data["similar_base_title"] = base_title

            update.message.reply_text(f"Похожие на '{base_title}':")
            show_more_similar(update, context)
        else:
            update.message.reply_text(
                f"Все похожие на '{base_title}' уже просмотрены или ничего не найдено."
            )

    except Exception as e:
        logger.error(f"Ошибка при поиске похожих: {e}")
        update.message.reply_text("Произошла ошибка при поиске похожих. Попробуйте ещё раз.")


def show_more_similar(update, context):
    """
    Выдаём следующую «партию» (3 фильма) из context.user_data["similar_movies"].
    Если ещё есть, добавляем кнопку «Показать ещё».
    """
    movies = context.user_data.get("similar_movies", [])
    idx = context.user_data.get("similar_index", 0)

    if idx >= len(movies):
        update.message.reply_text("Больше похожих фильмов нет.")
        return

    end_idx = idx + 3
    for m in movies[idx:end_idx]:
        send_movie_info(update, m)

    context.user_data["similar_index"] = end_idx

    if end_idx < len(movies):
        keyboard = [[InlineKeyboardButton("Показать ещё", callback_data="similar_more")]]
        reply_markup = InlineKeyboardMarkup(keyboard)
        update.message.reply_text("Показать ещё похожие фильмы?", reply_markup=reply_markup)
    else:
        update.message.reply_text("Это все похожие фильмы по данному запросу.")


# ------------------------------------------------------------------------------
# 3. Помощники (TMDB)
# ------------------------------------------------------------------------------
def get_movies_by_title(title):
    url = f"{TMDB_BASE_URL}/search/movie"
    params = {
        "api_key": TMDB_API_KEY,
        "language": "ru-RU",
        "query": title
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data.get("results", [])

def get_movies_by_genre(genre_name, years_range=None):
    genre_id = find_genre_id(genre_name)
    if not genre_id:
        return []

    params = {
        "api_key": TMDB_API_KEY,
        "language": "ru-RU",
        "sort_by": "popularity.desc",
        "with_genres": genre_id,
    }

    if years_range:
        start_year_str, end_year_str = years_range
        params["primary_release_date.gte"] = f"{start_year_str}-01-01"
        params["primary_release_date.lte"] = f"{end_year_str}-12-31"

    url = f"{TMDB_BASE_URL}/discover/movie"
    response = requests.get(url, params=params)
    data = response.json()
    return data.get("results", [])

def get_similar_movies(movie_id):
    """
    Запрашиваем фильмы, похожие на movie_id (GET /movie/{movie_id}/similar).
    """
    url = f"{TMDB_BASE_URL}/movie/{movie_id}/similar"
    params = {
        "api_key": TMDB_API_KEY,
        "language": "ru-RU",
        "page": 1
    }
    resp = requests.get(url, params=params)
    data = resp.json()
    return data.get("results", [])

def find_genre_id(genre_name):
    genres = get_genres()
    lower_name = genre_name.lower()
    for g in genres:
        if lower_name in g["name"].lower():
            return g["id"]
    return None

def get_genres():
    url = f"{TMDB_BASE_URL}/genre/movie/list"
    params = {
        "api_key": TMDB_API_KEY,
        "language": "ru-RU"
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data.get("genres", [])

def get_movie_details(movie_id):
    url = f"{TMDB_BASE_URL}/movie/{movie_id}"
    params = {
        "api_key": TMDB_API_KEY,
        "language": "ru-RU"
    }
    resp = requests.get(url, params=params)
    if resp.status_code == 200:
        return resp.json()
    else:
        return None


# ------------------------------------------------------------------------------
# show_more_movies: «Показать ещё» для поиска по жанру
# ------------------------------------------------------------------------------
def show_more_movies(update, context):
    movies = context.user_data.get("movies", [])
    next_index = context.user_data.get("next_index", 0)

    if next_index >= len(movies):
        update.message.reply_text("Больше фильмов нет по этому запросу.")
        return

    end_index = next_index + 3
    for m in movies[next_index:end_index]:
        send_movie_info(update, m)

    context.user_data["next_index"] = end_index

    if end_index < len(movies):
        keyboard = [[InlineKeyboardButton("Показать ещё", callback_data="show_more")]]
        reply_markup = InlineKeyboardMarkup(keyboard)
        update.message.reply_text(
            "Показать ещё фильмы?",
            reply_markup=reply_markup
        )
    else:
        update.message.reply_text("Это все фильмы по данному запросу.")


# ------------------------------------------------------------------------------
# 4. Обработка inline-кнопок
# ------------------------------------------------------------------------------
def handle_callback(update, context):
    query = update.callback_query
    query.answer()

    data = query.data
    if data == "show_more":
        # Показать ещё (по жанру)
        temp_update = create_temp_update_for_callback(query)
        show_more_movies(temp_update, context)

    elif data == "similar_more":
        # Показать ещё (по похожим)
        temp_update = create_temp_update_for_callback(query)
        show_more_similar(temp_update, context)

    elif data.startswith("watched_"):
        # "Я посмотрел"
        movie_id_str = data.split("_", 1)[1]
        try:
            movie_id = int(movie_id_str)
            user_id = query.from_user.id
            add_watched_movie(user_id, movie_id)
            query.message.reply_text("Отлично, записал, что вы посмотрели этот фильм!")
        except ValueError:
            logger.error("Некорректный movie_id при callback watched_: " + movie_id_str)

def create_temp_update_for_callback(query):
    class TempUpdate:
        def __init__(self, msg):
            self.message = msg
    return TempUpdate(query.message)


# ------------------------------------------------------------------------------
# 5. Отправка постера + описания (+ кнопка "Я посмотрел")
# ------------------------------------------------------------------------------
def send_movie_info(update, movie):
    title = movie.get("title", "Без названия")
    release_date = movie.get("release_date", "N/A")
    overview = movie.get("overview", "Нет описания :(")
    poster_path = movie.get("poster_path")
    vote_average = movie.get("vote_average") or 0.0
    movie_id = movie.get("id")

    caption_text = (
        f"<b>{title}</b>\n"
        f"Дата выхода: {release_date}\n"
        f"Рейтинг: {vote_average:.1f}/10\n\n"
        f"{overview}"
    )

    watch_button = InlineKeyboardButton("Я посмотрел", callback_data=f"watched_{movie_id}")
    keyboard = [[watch_button]]
    reply_markup = InlineKeyboardMarkup(keyboard)

    if poster_path:
        poster_url = f"{IMAGE_BASE_URL}{poster_path}"
        update.message.reply_photo(
            photo=poster_url,
            caption=caption_text,
            parse_mode="HTML",
            reply_markup=reply_markup
        )
    else:
        update.message.reply_text(
            caption_text,
            parse_mode="HTML",
            reply_markup=reply_markup
        )


def send_watched_movie_info(update, movie):
    title = movie.get("title", "Без названия")
    release_date = movie.get("release_date", "N/A")
    text = f"— {title} ({release_date})"
    update.message.reply_text(text)


# ------------------------------------------------------------------------------
# 6. Обработчик «всех прочих» сообщений
# ------------------------------------------------------------------------------
def handle_text(update, context):
    update.message.reply_text(
        "Используйте команды:\n"
        "/search_title <название фильма>\n"
        "/search_genre <жанр> [год или диапазон годов]\n"
        "/similar_title <название> — фильмы, похожие на...\n"
        "/my_watched — все просмотренные фильмы\n"
        "/clear_watched — очистить список просмотренных\n\n"
        "Пример: /search_genre боевик 2015-2017\n"
        "Пример: /search_title Аватар\n"
        "Пример: /similar_title Аватар"
    )

# ------------------------------------------------------------------------------
# Запуск бота
# ------------------------------------------------------------------------------
def run_bot():
    init_db()
    updater = Updater(TELEGRAM_TOKEN, use_context=True)
    dp = updater.dispatcher

    dp.add_handler(CommandHandler("start", start_command))
    dp.add_handler(CommandHandler("search_title", search_title_command))
    dp.add_handler(CommandHandler("search_genre", search_genre_command))
    dp.add_handler(CommandHandler("similar_title", similar_title_command))
    dp.add_handler(CommandHandler("my_watched", my_watched_command))
    dp.add_handler(CommandHandler("clear_watched", clear_watched_command))

    dp.add_handler(CallbackQueryHandler(handle_callback))
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_text))

    updater.start_polling()
    print("Бот запущен. Нажмите Ctrl+C (или остановите ячейку), чтобы завершить.")

In [4]:
import threading

bot_thread = threading.Thread(target=run_bot, daemon=True)
bot_thread.start()

2025-01-16 17:49:40,900 - apscheduler.scheduler - INFO - Scheduler started


Бот запущен. Нажмите Ctrl+C (или остановите ячейку), чтобы завершить.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [2]:
credits.columns=['id','title','cast','crew']
movies = movies.merge(credits,on='id')

In [4]:
movies = movies[['id','original_title','overview','genres','cast','keywords','crew']]
movies.head()

,id,original_title,overview,genres,cast,keywords,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [7]:
movies.dropna(inplace=True)
movies.isnull().sum()

id                0
original_title    0
overview          0
genres            0
cast              0
keywords          0
crew              0
dtype: int64

In [8]:
import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

#function for extracting top(first) 3 actors from the movie
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter !=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

movies['cast'] = movies['cast'].apply(convert3)

#function to fetch the director of movie from the crew column
def fetch_director(obj):
    L= []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

movies['crew'] = movies['crew'].apply(fetch_director)
movies.head()

,id,original_title,overview,genres,cast,keywords,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[culture clash, future, space war, space colon...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[Johnny Depp, Orlando Bloom, Keira Knightley]","[ocean, drug abuse, exotic island, east india ...",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[Daniel Craig, Christoph Waltz, Léa Seydoux]","[spy, based on novel, secret agent, sequel, mi...",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[Christian Bale, Michael Caine, Gary Oldman]","[dc comics, crime fighter, terrorist, secret i...",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[Taylor Kitsch, Lynn Collins, Samantha Morton]","[based on novel, mars, medallion, space travel...",[Andrew Stanton]


In [9]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
        
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'crew', 'genres']

for feature in features:
    movies[feature] = movies[feature].apply(clean_data)
    
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' +' '.join(x['crew']) + ' ' + ' '.join(x['genres'])
movies['soup'] = movies.apply(create_soup, axis=1)

In [10]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(movies['soup'])

# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)

# Reset index of our main DataFrame and construct reverse mapping
movies = movies.reset_index()
indices = pd.Series(movies.index, index=movies['original_title'])

# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies['original_title'].iloc[movie_indices]

In [11]:
get_recommendations('The Dark Knight Rises', cosine_sim)

65               The Dark Knight
119                Batman Begins
4635    Amidst the Devil's Wings
3331                 Harry Brown
3818                    Defendor
4098                 Harsh Times
210               Batman & Robin
3358                 In Too Deep
1503                      Takers
1986                      Faster
Name: original_title, dtype: object